In [ ]:
%load_ext autoreload

In [ ]:
import os as _os
_os.chdir(_os.environ['PROJECT_ROOT'])
_os.path.realpath(_os.path.curdir)

#### Imports

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
from lib.pandas_util import idxwhere, aligned_index, align_indexes, invert_mapping
import lib.thisproject.data
import matplotlib as mpl
import lib.plot
import statsmodels as sm
from statsmodels.stats.multitest import fdrcorrection
from tqdm import tqdm
import subprocess
from tempfile import mkstemp
import time
import subprocess
from itertools import chain
import os
from itertools import product
from mpl_toolkits.axes_grid1 import make_axes_locatable
import sfacts as sf
import lib.thisproject.data

In [ ]:
species_depth = lib.thisproject.data.load_species_depth("data/group/xjin_hmp2/r.proc.gtpro.species_depth.tsv")
# species_depth_subset = lib.thisproject.data.load_species_depth(path['all_species_depth_subset'])
rabund = species_depth.apply(lambda x: x / x.sum(), axis=1)
# rabund_subset = species_depth_subset.apply(lambda x: x / x.sum(), axis=1)

n_species = 40
top_species = (species_depth > 1e-3).sum().sort_values(ascending=False).head(n_species).index

fig, axs = plt.subplots(n_species, figsize=(5, 0.25 * n_species), sharex=True, sharey=True)

bins = np.logspace(-3, 4, num=51)

for species_id, ax in zip(top_species, axs):
    # ax.hist(rabund_subset[species_id], bins=bins, alpha=0.7)
    ax.hist(species_depth[species_id], bins=bins, alpha=0.7)
    ax.set_xscale('log')
    prevalence = (species_depth[species_id] > 1e-3).mean()
    ax.set_title("")
    # ax.set_xticks()
    # ax.set_yticks()
    ax.yaxis.set_visible(False)
    ax.xaxis.set_visible(False)
    ax.patch.set_alpha(0.0)
    for spine in ['left', 'right', 'top', 'bottom']:
        ax.spines[spine].set_visible(False)
    ax.annotate(f'{species_id} ({prevalence:0.0%})', xy=(0.05, 0.1), ha='left', xycoords="axes fraction")
    ax.set_xlim(left=1e-7)
    ax.set_ylim(top=300)
    
ax.xaxis.set_visible(True)
ax.spines['bottom'].set_visible(True)

# fig.subplots_adjust(hspace=-0.75)

In [ ]:
num_samples_with_depth_gt_half = (species_depth > 0.5).sum()

bins = [0] + list(np.logspace(0, 4))
plt.hist(num_samples_with_depth_gt1, bins=bins)
plt.xscale('symlog', linthresh=1.0, linscale=0.1)

In [ ]:
# species_id = '100099'  # '100236'

species_taxonomy = lib.thisproject.data.load_species_taxonomy("ref/gtpro/species_taxonomy_ext.tsv")
species_taxonomy.assign(num_samples_with_depth_gt_half=num_samples_with_depth_gt_half).fillna(0).sort_values('num_samples_with_depth_gt_half', ascending=False).head(10).drop(columns=['taxonomy_string'])

In [ ]:
num_strains_raw = {}
for species_id in num_samples_with_depth_gt_half.index:
    try:
        _spgc_meta = pd.read_table(f"data/group/xjin_hmp2/species/sp-{species_id}/r.proc.gtpro.sfacts-fit.gene99-v22-agg75.spgc_specgene-ref-t25-p95_ss-all_t-30_thresh-corr200-depth250.strain_meta.tsv", index_col='strain').rename(str)
    except FileNotFoundError as err:
        print(err)
        continue
    num_strains_raw[species_id] = _spgc_meta.shape[0]
    
num_strains_raw = pd.Series(num_strains_raw)

In [ ]:
d = species_taxonomy.assign(num_samples_with_depth_gt_half=num_samples_with_depth_gt_half, num_strains_raw=num_strains_raw).fillna(0)
plt.scatter('num_samples_with_depth_gt_half', 'num_strains_raw', data=d)
plt.yscale('log')
plt.xscale('log')
plt.xlabel('num_samples_with_depth_gt_half')
plt.ylabel('num_strains_raw')

In [ ]:
num_strains_filt = {}
num_strains_raw = {}

for species_id in tqdm(num_samples_with_depth_gt_half.index):
    try:
        _spgc_meta = pd.read_table(f"data/group/xjin_hmp2/species/sp-{species_id}/r.proc.gtpro.sfacts-fit.gene99-v22-agg75.spgc_specgene-ref-t25-p95_ss-all_t-30_thresh-corr200-depth250.strain_meta.tsv", index_col='strain').rename(str)
    except FileNotFoundError as err:
        pass # print(species_id, "File missing.")
    else:
        num_strains_raw[species_id] = _spgc_meta.shape[0]
        x = _spgc_meta[_spgc_meta.species_gene_frac > 0.9].num_genes
        if len(x) < 1:
            pass # print(species_id, "No strains have >90% of species genes.")
            num_strains_filt[species_id] = 0
        else:
            _df, _loc, _scale = sp.stats.t.fit(x.values, fix_df=2)
            _dist0 = sp.stats.t(_df, _loc, _scale)
            _dist1 = sp.stats.norm(_loc, _scale)
            thresh_max_num_uhgg_genes = _dist1.ppf(0.999)
            thresh_min_num_uhgg_genes = _dist1.ppf(0.001)
            num_strains_filt[species_id] = (
                (_spgc_meta.sum_depth > 5)
                & (_spgc_meta.species_gene_frac > 0.9)
                & (_spgc_meta.num_genes <= thresh_max_num_uhgg_genes)
                & (_spgc_meta.num_genes >= thresh_min_num_uhgg_genes)
            ).sum()
    
num_strains_filt = pd.Series(num_strains_filt)
num_strains_raw = pd.Series(num_strains_raw)

In [ ]:
num_strains_filt.sum()

In [ ]:
d0 = (
    pd.DataFrame(dict(
        num_strains_filt=num_strains_filt,
        num_strains_raw=num_strains_raw,
        num_samples_with_depth_gt_half=num_samples_with_depth_gt_half,
    ))
    .fillna(0)
    .join(species_taxonomy)
    .sort_values('num_strains_filt', ascending=False)
)

o__order = d0['o__'].value_counts().index.values
top_o = o__order[:5]
o__palette = lib.plot.construct_ordered_palette(top_o)
plt.scatter('num_strains_raw', 'num_strains_filt', data=d0, c='num_samples_with_depth_gt_half', norm=mpl.colors.SymLogNorm(1.0))
plt.colorbar()
plt.xlabel('num_strains_raw')
plt.ylabel('num_strains_filt')
plt.xscale('symlog')
plt.yscale('symlog')

In [ ]:
d0 = (
    pd.DataFrame(dict(
        num_strains_filt=num_strains_filt,
        num_strains_raw=num_strains_raw,
        num_samples_with_depth_gt_half=num_samples_with_depth_gt_half,
    ))
    .join(species_taxonomy)
    .sort_values('num_strains_filt', ascending=False)
)

o__order = d0['o__'].value_counts().index.values
top_o = o__order[:10]
o__palette = lib.plot.construct_ordered_palette(top_o, cm='tab20')

for o__ in o__order:
    d1 = d0[d0.o__ == o__]
    plt.scatter('num_strains_raw', 'num_strains_filt', data=d1, c=o__palette[o__], label=o__)
plt.xlabel('num_strains_raw')
plt.ylabel('num_strains_filt')
plt.xscale('symlog')
plt.yscale('symlog')
plt.legend(bbox_to_anchor=(1, 1))

d

In [ ]:
d0 = (
    pd.DataFrame(dict(
        num_strains_filt=num_strains_filt,
        num_strains_raw=num_strains_raw,
        num_samples_with_depth_gt_half=num_samples_with_depth_gt_half,
    ))
    .assign(num_filtered_out=lambda x: x.num_strains_raw - x.num_strains_filt)
    .join(species_taxonomy)
    .sort_values('num_strains_filt', ascending=False)
)

d0.sort_values('num_filtered_out', ascending=False)

In [ ]:
d0.sort_values('num_strains_filt', ascending=False)